In [15]:
!pip install keras-tuner

In [16]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [17]:
fm = keras.datasets.fashion_mnist

In [18]:
(train_images,train_labels),(test_images,test_labels) = fm.load_data()

In [19]:
train_images = train_images/255.0
test_images = test_images/255.0

In [20]:
train_images[0].shape

(28, 28)

In [21]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [22]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [23]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters 

In [24]:
tuner_search = RandomSearch(build_model,objective = 'val_accuracy',max_trials = 5, directory = 'output',project_name = 'Mnist Fashion')

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


In [25]:
tuner_search.search(train_images,train_labels,validation_split = 0.1, epochs = 3)

INFO:tensorflow:Oracle triggered exit


In [26]:
model = tuner_search.get_best_models(num_models=1)[0]

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 48)        480       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        57648     
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 48)                1115184   
                                                                 
 dense_1 (Dense)             (None, 10)                490       
                                                                 
Total params: 1,173,802
Trainable params: 1,173,802
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1944 - accuracy: 0.9294 - val_loss: 0.2598 - val_accuracy: 0.9103
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1480 - accuracy: 0.9449 - val_loss: 0.2356 - val_accuracy: 0.9178
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1112 - accuracy: 0.9592 - val_loss: 0.2924 - val_accuracy: 0.9178
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0837 - accuracy: 0.9687 - val_loss: 0.3326 - val_accuracy: 0.9093
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0652 - accuracy: 0.9762 - val_loss: 0.3227 - val_accuracy: 0.9117
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0488 - accuracy: 0.9821 - val_loss: 0.4007 - val_accuracy: 0.9098
Epoch 10/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0413 - accuracy: 0.9849 - val_loss: 0.4021 - val_accuracy